In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import heapq

In [2]:
df=pd.read_csv('ratings.csv')

In [36]:
user_movie_rating = df.pivot_table(index='userId', columns='movieId', values='rating')

In [55]:
dist=[]
klen=[]

In [56]:
for j in tqdm(range(1,len(user_movie_rating.index)+1)):
    dist_temp=[]
    klen_temp=[]
    for i in range(1,len(user_movie_rating.index)+1):
        user1_rating=user_movie_rating.iloc[j-1][user_movie_rating.iloc[j-1].isna()==False]
        user1_movieId=list(user_movie_rating.iloc[j-1][user_movie_rating.iloc[j-1].isna()==False].index)
        user_i_rating=user_movie_rating.iloc[i-1][user_movie_rating.iloc[i-1].isna()==False]
        user_i_1_rating=user_i_rating[user_i_rating.index.isin(user1_movieId)]
        user_i_1_movieId=list(user_i_1_rating.index)
        user1_i_rating=user1_rating[user1_rating.index.isin(user_i_1_movieId)]
        a=sum(user1_i_rating*user_i_1_rating)
        b=np.sqrt(sum(np.square(user_i_1_rating)))
        c=np.sqrt(sum(np.square(user1_rating)))
        k=a/(b*c)
        klen_temp.append(k)
        dist_temp.append(np.cos(k))
    klen.append(klen_temp)
    dist.append(dist_temp)

100%|██████████| 11/11 [00:00<00:00, 17.72it/s]


In [57]:
df1=pd.DataFrame(klen)

In [58]:
df1

,0,1,2,3,4,5,6,7,8,9,10
0,1.000000,0.837808,0.803917,0.535504,0.622752,0.547105,0.490290,NaN,0.392232,NaN,0.535504
1,0.792963,1.000000,0.867363,0.250282,0.457034,0.294437,0.378633,0.249757,0.316145,0.249757,0.313091
2,0.866111,0.898717,1.000000,0.400320,0.660578,0.306186,0.319444,0.216506,0.283473,0.216506,0.471405
3,0.517932,0.369368,0.305566,1.000000,0.930484,0.996024,NaN,NaN,0.089087,NaN,1.000000
4,0.700481,0.466082,0.558064,0.913347,1.000000,0.878785,0.170872,0.170872,0.269376,0.170872,0.917325
5,0.585595,0.441899,0.440225,0.984374,0.978839,1.000000,NaN,NaN,NaN,NaN,0.984374
6,0.189832,0.502592,0.476383,NaN,0.474579,NaN,1.000000,0.963465,0.969398,0.963465,NaN
7,NaN,0.519705,0.518805,NaN,0.386695,NaN,0.981308,1.000000,0.996327,1.000000,NaN
8,0.092450,0.435143,0.303822,0.092450,0.223178,NaN,0.983123,0.992061,1.000000,0.992061,0.092450
9,NaN,0.519705,0.518805,NaN,0.386695,NaN,0.981308,1.000000,0.996327,1.000000,NaN


In [59]:
df2=pd.DataFrame(dist)

In [60]:
df2

,0,1,2,3,4,5,6,7,8,9,10
0,0.540302,0.669093,0.693892,0.860011,0.812276,0.854034,0.882196,NaN,0.924058,NaN,0.860011
1,0.701737,0.540302,0.646840,0.968843,0.897365,0.956966,0.929171,0.968972,0.950441,0.968972,0.951386
2,0.647794,0.622614,0.540302,0.920936,0.789638,0.953490,0.949410,0.976654,0.960090,0.976654,0.890931
3,0.868845,0.932556,0.953677,0.540302,0.597446,0.543644,NaN,NaN,0.996034,NaN,0.540302
4,0.764532,0.893336,0.848282,0.611100,0.540302,0.638087,0.985437,0.985437,0.963937,0.985437,0.607946
5,0.833383,0.903941,0.904656,0.553385,0.557987,0.540302,NaN,NaN,NaN,NaN,0.553385
6,0.982036,0.876337,0.888659,NaN,0.889485,NaN,0.540302,0.570678,0.565796,0.570678,NaN
7,NaN,0.867966,0.868412,NaN,0.926161,NaN,0.555935,0.540302,0.543389,0.540302,NaN
8,0.995730,0.906810,0.954200,0.995730,0.975199,NaN,0.554426,0.546966,0.540302,0.546966,0.995730
9,NaN,0.867966,0.868412,NaN,0.926161,NaN,0.555935,0.540302,0.543389,0.540302,NaN


In [72]:
df1.fillna(0,inplace=True)

In [73]:
df1.to_csv('cos_similarity_improved.csv', index=False, header=False)

In [11]:
# df2.to_csv('angle_similarity_improved.csv', index=False, header=False)

In [62]:
similarity_df=pd.read_csv('cos_similarity_improved.csv',header=None)

In [63]:
similarity_df.fillna(0.0,inplace=True)

In [64]:
user_similarity_dict={}
for i in tqdm(range(similarity_df.shape[0])):
    sorted_similarity=heapq.nlargest(10,similarity_df.iloc[i])[1:]
    dict1={}
    for j in sorted_similarity:
        dict1[similarity_df.iloc[i][similarity_df.iloc[i]==j].index.values[0]+1]=j
    user_similarity_dict[i+1]=dict1

100%|██████████| 11/11 [00:00<00:00, 130.76it/s]


In [65]:
abs_error_sum=0
root_square_sum=0
count=0

for i in tqdm(user_movie_rating.index):
    non_null_movies=list(user_movie_rating.loc[i][user_movie_rating.loc[i].isna()==False].index)
    k0=user_movie_rating.loc[list(user_similarity_dict[i].keys())[0]]
    k0=k0[k0.index.isin(non_null_movies)]
    k0.fillna(0,inplace=True)
    k1=user_movie_rating.loc[list(user_similarity_dict[i].keys())[1]]
    k1=k1[k1.index.isin(non_null_movies)]
    k1.fillna(0,inplace=True)
    k2=user_movie_rating.loc[list(user_similarity_dict[i].keys())[2]]
    k2=k2[k2.index.isin(non_null_movies)]
    k2.fillna(0,inplace=True)
    a=list(user_similarity_dict[i].values())[0]
    b=list(user_similarity_dict[i].values())[1]
    c=list(user_similarity_dict[i].values())[2]
    predicted_data=((k0*a+k1*b+k2*c)/(a+b+c))
    predicted_data=np.ceil(predicted_data*2)/2
    predicted_data=predicted_data.replace(6.0,5.0)
    predicted_data=predicted_data.replace(5.5,5.0)
    actual_data=user_movie_rating.loc[i][user_movie_rating.loc[i].isna()==False]
    abs_error_sum=abs_error_sum+sum(np.abs(predicted_data-actual_data))
    root_square_sum=root_square_sum+sum(np.square(predicted_data-actual_data))
    count=count+len(non_null_movies)

  0%|          | 0/11 [00:00<?, ?it/s]


KeyError: 0

In [66]:
np.sqrt(root_square_sum/count)

ZeroDivisionError: division by zero

In [67]:
abs_error_sum/count

ZeroDivisionError: division by zero